# Capstone Project-The Battle of the Neighborhoods (Week 1)

## Table of contents
* Introduction: Business Problem
* Data
* Methodology
* Analysis
* Results and Discussion
* Conclusion

## Introduction

1.1.Background

This data science project aims to help my friend who is planning to shift Brooklyn, New York. He is interested to start a coffee shop in one of the neighbourhoods of Brooklyn. As a data scientist I would like to help my friend by doing survey of Brooklyn neighbourhoods and suggest him right place to start his business. Each neighbourhoodneed to be analysed for no or less number of existing coffee shops.

1.2.Business Problem

This project aims to predict more suitable neighbourhood in Brooklyn for a coffee shop in terms of the no. of existing coffee shops and most trending.It is required to explore, segment, and cluster the neighbourhoods in the city of Brooklyn and find out location with no or less number of coffee shops.

1.3.Audience

For anyone who wants to explore options of starting new business in US or any other place , this kind of analysis can be useful. The methodology will be the same which is used here in this project.


## Data

Data Sources

New York Neighborhood data has a total of 5 boroughs and 306 neighborhoods. In order to segment the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the latitude and longitude coordinates of each neighborhood. 
This dataset exists on the web here - https://geo.nyu.edu/catalog/nyu_2451_34572


The data will be used as follows:

Using geopy library get the list of the latitude and longitude values of New York City
Slice the original dataframe and create a new dataframe of the Brooklyn neighborhoods with their latitude and longitude
Using Foursquare explore all neighborhood in data frame.
Use Foursquare and geopy data to map top 5 venues for Brooklyn neighborhoods and clustered in groups
 


Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##  Download and Explore Dataset

!wget -q -O 'data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

In [14]:
with open('data.json') as json_data:
    data = json.load(json_data)
neigh_data = data['features']

In [21]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhood = pd.DataFrame(columns=column_names)

In [22]:
for neigh in neigh_data:
    borough = neigh['properties']['borough'] 
    neigh_name = neigh['properties']['name']
        
    neigh_latlon = neigh['geometry']['coordinates']
    neigh_lat = neigh_latlon[1]
    neigh_lon = neigh_latlon[0]
    
    neighborhood = neighborhood.append({'Borough': borough,'Neighborhood': neigh_name,'Latitude': neigh_lat,'Longitude': neigh_lon}, ignore_index=True)

In [24]:
brooklyn_data= neighborhood[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [27]:
address = 'Brooklyn, NY'
geolocator=Nominatim(user_agent='ny_explorer')
location=geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


In [ ]:
**Folium** is a great visualization library.Use it to exlore Brooklyn and the neighborhood and its respective borough.

In [30]:
# create map of Brooklyn using latitude and longitude values
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

## Use Foursquare API to find out Venues details

In [ ]:
CLIENT_ID = 'T4KVRWB54PTAC5SWRSNENFQVDQJ1EY0POYDV1IIYCUF3O2RP' # your Foursquare ID
CLIENT_SECRET = '1PLNALU0ZUQ3OVFW43ZHNBDLFWRN4GULOEPSWFBGCEE3RKYR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)